In [165]:
from credentials import *
import tweepy
import re

In [1]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True) # do not miss this line


In [4]:
# API setup

def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access keys provided.
    """
    
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    
    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [3]:
# Create an extractor object:
api = twitter_setup()

# Create a tweet list as follows:
query = '"ali caldwell"'
tweets = api.search(q=query)
print("Number of tweets extracted: {}".format(len(tweets)))
x = [i._json for i in tweets]

Number of tweets extracted: 15


In [6]:
# Create an extractor object:
api = twitter_setup()

# Create a tweet list as follows:
screen_name = "iAmAliCaldwell"
tweets = api.user_timeline(screen_name=screen_name)
print("Number of tweets extracted: {}".format(len(tweets)))
x = [i._json for i in tweets]

Number of tweets extracted: 20


In [91]:
all_data = pd.DataFrame(x)

In [4]:
#TweetMiner function from Mike Roman

class TweetMiner(object):

    
    def __init__(self, api, result_limit = 20):
        
        self.api = api        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="iAmAliCaldwell", mine_retweets=True, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets)
                statuses = [ x._json for x in statuses]
            else:
                statuses   =   self.api.user_timeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [x._json for x in statuses]
                
            for item in statuses:
                mined = item
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
    def mine_mentions(self, q='"ali caldwell"', mine_retweets=True, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.search(q=query, count=self.result_limit, include_rts=mine_retweets)
                statuses = [ x._json for x in statuses]
            else:
                statuses   =   self.api.search(q=query, count=self.result_limit, include_rts=mine_retweets)
                statuses = [x._json for x in statuses]
                
            for item in statuses:
                mined = item
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [8]:
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(api, result_limit=200)

In [152]:
ali = miner.mine_user_tweets(user="iAmAliCaldwell")

In [153]:
ali_tweets = pd.DataFrame(ali)

In [154]:
ali_tweets.to_csv('iamali_tweets.csv', index=False)

In [150]:
# ali_tweets = pd.read_csv('./iamali_tweets.csv')

TypeError: parser_f() got an unexpected keyword argument 'index'

In [155]:
ali_tweets['created_at'] = pd.to_datetime(ali_tweets['created_at'])
aliT = ali_tweets['created_at'].copy()

In [158]:
ali_tweets.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'extended_entities', 'favorite_count', 'favorited', 'geo', 'id',
       'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'text', 'truncated', 'user'],
      dtype='object')

In [160]:
ali_tweets.drop(['contributors', 'id_str', 'possibly_sensitive', 'source', 'truncated'], axis=1, inplace=True)

In [23]:
ali_tweets

NameError: name 'ali_tweets' is not defined

In [166]:
ali_tweets['text'].apply(handle_emojis)

0       I love you for loving me! ❤️\nY’all ready for ...
1       R.I.P Queen👑 @ArethaFranklin \n😥❤❤❤❤❤\nhttps:/...
2       Aretha Franklin - Natural Woman - (Ali Caldwel...
3       Omg! YES! Thank you! 🙏🏾  🙌🏾 New music coming s...
4       You never know what someone is going through! ...
5       Mannnnn another young soul gone from drug over...
6                                               Excited 🤗
7       Shooting a commercial tomorrow! Can’t wait to ...
8                   Going LIVE on my IG tomorrow at 630PM
9                               🍋 https://t.co/kZNoU8806t
10      Consistency has CHANGED MY LIFE!\nFind that PU...
11      RT @JasmineWills94: Omg she So Gorgeous @iamAl...
12      🙈🙈🙈🙈🙈 Google better SPEAK THAT INTO MY LIFE!! ...
13                                       Google funny AF!
14      Thank you for all your messages and prayers an...
15                      Btw guys I’m feeling a lot better
16      If you’re brown skin,\nFair skin,\nDark skin,\...
17      Feelin

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(ali_tweets['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('cancer https', 133),
 ('ali caldwell', 86),
 ('thank https', 70),
 ('https gbf5fxjehpyou', 38),
 ('cancer https gbf5fxjehpyou', 38),
 ('iamalicaldwell https', 36),
 ('teammiley https', 32),
 ('new single', 30),
 ('gt gt', 29),
 ('iamalicaldwell love', 29),
 ('make sure', 27),
 ('alicaldwell https', 25),
 ('single loved', 25),
 ('nbcthevoice iamalicaldwell', 25),
 ('new music', 24),
 ('love https', 23),
 ('thevoice https', 22),
 ('video https', 21),
 ('nbcthevoice https', 21),
 ('new single loved', 21)]

In [155]:
#cleaning text data using textacy#cleaning 
from textacy.preprocess import preprocess_text

tweet_text = ali_tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [156]:
print (tweet_text[1:8])

['R.I.P Queen👑 @ArethaFranklin \n😥❤❤❤❤❤\nhttps://t.co/rb8OAN4mco'
 'Aretha Franklin - Natural Woman - (Ali Caldwell &amp; Andre Sims Tribute): https://t.co/d2AgSCwHt9 via @YouTube'
 'Omg! YES! Thank you! 🙏🏾  🙌🏾 New music coming soon❤️ https://t.co/riQZl7YASp'
 'You never know what someone is going through! \n\nBE KIND!\nLove MORE!\nYou just NEVER KNOW!'
 'Mannnnn another young soul gone from drug overdose... so sad! 😢 Reading his last tweets got me so emotional! 🙏🏾 Pra… https://t.co/M6gecmcZRY'
 'Excited 🤗'
 'Shooting a commercial tomorrow! Can’t wait to share the news with you all! 🙏🏾']


In [157]:
print (clean_text[1:8])


['r i p queen👑 arethafranklin 😥❤❤❤❤❤\n url', 'aretha franklin natural woman ali caldwell andre sims tribute url via youtube', 'omg yes thank you 🙏🏾 🙌🏾 new music coming soon❤️ url', 'you never know what someone is going through be kind love more you just never know', 'mannnnn another young soul gone from drug overdose so sad 😢 reading his last tweets got me so emotional 🙏🏾 pra url', 'excited 🤗', 'shooting a commercial tomorrow can t wait to share the news with you all 🙏🏾']


Looking at mentions using search "ali caldwell"

In [12]:
# Result limit == count parameter from our GetUserTimeline()
miner_search = TweetMiner(api, result_limit=200)

In [13]:
ali_search = miner_search.mine_mentions(q='"ali caldwell"')

In [14]:
ali_search_df = pd.DataFrame(ali_search)

In [1]:
ali_search_df

NameError: name 'ali_search_df' is not defined

In [7]:
import pandas as pd

In [18]:
from get_old_tweets import got3 as got

In [20]:
def query_hist(query, start_date, end_date):
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch(query).setSince(start_date).setUntil(end_date)
    tweets = got.manager.TweetManager.getTweets(tweet_criteria)

    for i in range(len(tweets)):
        d = {'index':tweets[i].date, 'text':tweets[i].text, 'id':tweets[i].id, 'username':tweets[i].username,
             'retweets':tweets[i].retweets, 'favorites':tweets[i].favorites,  'mentions':tweets[i].mentions,
             'hashtags':tweets[i].hashtags, 'geo':tweets[i].geo, 'permalink':tweets[i].permalink}
        if i == 0:
            df = pd.DataFrame.from_dict(d,orient='index').T
            df.index = df['index']
            df = df.drop('index', axis=1)
        else:
            df2 = pd.DataFrame.from_dict(d,orient='index').T
            df2.index = df2['index']
            df2 = df2.drop('index', axis=1)
            df = df.append(df2)

    
    return df

In [21]:
query = '#alicaldwell'
start_date = '2018-05-01'
end_date = '2018-08-31'

In [22]:
df = query_hist(query, start_date, end_date)
df.head()

,text,id,username,retweets,favorites,mentions,hashtags,geo,permalink
index,,,,,,,,,
2018-08-21 08:24:16,RT @BoetPrince : RT @fanseize : RT @BoetPrince...,1031879656994160641,,1,1,@BoetPrince @fanseize @BoetPrince @iamAliCaldw...,#NewVideo #LINKINBIO #AliCaldwell #Bratislava ...,,https://twitter.com/fanseize/status/1031879656...
2018-08-21 06:24:32,RT @BoetPrince : RT @iamAliCaldwell : It's Off...,1031849525932183552,,1,1,@BoetPrince @iamAliCaldwell @iamalicaldwell @vevo,#NewVideo #LINKINBIO #AliCaldwell #Bratislava ...,,https://twitter.com/fanseize/status/1031849525...
2018-08-19 23:54:23,@TheFourOnFOX #AliCaldwell sounds like she is ...,1031388953394536451,,0,0,@TheFourOnFOX,#AliCaldwell,,https://twitter.com/lovettstough/status/103138...
2018-08-17 10:41:24,I had a fun interview during my @bmi Takeover ...,1030464616252538880,,7,38,@bmi @bmi @ogrant10 @bmi,#BMILive #igtv #Alicaldwell #BMI #takeover #BM...,,https://twitter.com/iamAliCaldwell/status/1030...
2018-08-17 05:48:47,Give us new music to enjoy @iamAliCaldwell . S...,1030390979096801280,,1,0,@iamAliCaldwell,#alicaldwell #thefour #thefourali,,https://twitter.com/bloodyhellp/status/1030390...


In [14]:
tweets = pd.read_csv('#alicaldwell.csv')

In [16]:
for i in range(len(tweets)):
    d = {'index':tweets[i].date, 'text':tweets[i].text, 'id':tweets[i].id, 'username':tweets[i].username,
         'retweets':tweets[i].retweets, 'favorites':tweets[i].favorites,  'mentions':tweets[i].mentions,
         'hashtags':tweets[i].hashtags, 'geo':tweets[i].geo, 'permalink':tweets[i].permalink}
    if i == 0:
        df = pd.DataFrame.from_dict(d,orient='index').T
        df.index = df['index']
        df = df.drop('index', axis=1)
    else:
        df2 = pd.DataFrame.from_dict(d,orient='index').T
        df2.index = df2['index']
        df2 = df2.drop('index', axis=1)
        df = df.append(df2)

KeyError: 0

In [12]:
pd.read_csv('#alicaldwell.csv')

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,"permalink,,,,,"
0,";2018-08-21 08:24;1;1;""RT @BoetPrince : RT @fa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,";2018-08-21 06:24;1;1;""RT @BoetPrince : RT @ia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,";2018-08-19 23:54;0;0;""@TheFourOnFOX #AliCaldw...",NaN,@TheFourOnFOX,#AliCaldwell,"1031388953394536451""""",https://twitter.com/lovettstough/status/103138...,NaN,NaN,NaN,NaN
3,";2018-08-17 10:41;7;38;""I had a fun interview ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,";2018-08-17 05:48;1;0;""Give us new music to en...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,";2018-08-13 15:33;0;2;""I simply refuse to watc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,";2018-08-13 14:03;0;3;""Tune in #RN ! We are li...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,";2018-08-12 16:00;5;30;""I will be taking over ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,";2018-08-11 12:18;3;14;""@iamAliCaldwell #Alica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,";2018-08-10 22:17;0;1;""http://www. NOCTURNALVI...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
